# Liquid argon as a Lennard-Jones fluid

We here study intermolecular interactions in liquid argon using the [Lennard-Jones interaction potential](https://en.wikipedia.org/wiki/Lennard-Jones_potential).
Experimental observations:

| Property                       | Symbol                  | Value           |
| ------------------------------ | ----------------------- | --------------- |
| Boiling point                  | $T_b$                   | **87.30 K**     |
| Heat of vaporization           | $\Delta H_{\text{vap}}$ | **6.53 kJ/mol** |
| Density of liquid, near ($T_b$)| $\rho$                  | **1.394 g/cm³** |

## Questions

1. What interactions hold liquid argon together?
1. Calculate the molar volume for argon and estimate the diameter
2. Assuming that $g(r)=1$ for $\sigma>r$ and zero otherwise, give an analytical expression for the average energy per argon molecule, i.e. $U/N$.
3. How strong should the attraction be to reproduce the experimental heat of vaporization? Is the value compatible with your answer to (1)?
4. Does the real $U/N$ depend on temperature. Why or why not.

## Average energy per particle
Solve the following integral with $u(r)$ set to the LJ potential. We note that the integral is zero from 0 to $\sigma$ since $g(r)=0$. This is not difficult to do by hand, but we also show how to symbolically solve this using `Sympy`.
$$
U/N = \frac{1}{2} \cdot 4\pi\rho \int_0^{\infty} g(r) u(r) r^2 dr
$$

In [1]:
from sympy import Eq, symbols, integrate, oo, Symbol, pi

r, sigma, epsilon, rho = symbols('r sigma epsilon rho', real=True)
u = 4 * epsilon * ((sigma / r)**12 - (sigma / r)**6)
avg_energy = 2 * pi * rho * integrate(u * r**2, (r, sigma, oo))

Eq(Symbol("U/N"), avg_energy) # pretty print

Eq(U/N, -16*pi*epsilon*rho*sigma**3/9)

## Heat of vaporization

The heat of evaporation is the energy needed to break away a molecule from the liquid and into the gas phase. If the gas phase is ideal, this is exactly the negative of the average energy per particle in the liquid.

In [2]:
from scipy.constants import Avogadro as N_A, pi

epsilon        = 1.1                                    # kJ / mol (fitted to match experiment)
rho            = 1.394                                  # g / ml
Mw             = 39.9                                   # g / mol
molar_volume   = Mw / rho                               # ml / mol
number_density = 1 / (molar_volume * 1e-3 * 1e27 / N_A) # 1 / angstrom^3
sigma          = (1 / number_density)**(1/3)            # angstrom
avg_energy     = -16 / 9 * pi * epsilon * number_density * sigma**3

print(f"Number density = {number_density:.2f} Å-3")
print(f"Molar volume   = {molar_volume:.2f} ml/mol")
print(f"Diameter       = {sigma:.1f} Å")
print(f"Avg. energy    = {avg_energy:.2f} kJ/mol per particle")
print(f"Heat of vap.   = {-avg_energy:.2f} kJ/mol")

Number density = 0.02 Å-3
Molar volume   = 28.62 ml/mol
Diameter       = 3.6 Å
Avg. energy    = -6.14 kJ/mol per particle
Heat of vap.   = 6.14 kJ/mol


## Trouton's rule

We can use [Trouton's rule](https://en.wikipedia.org/wiki/Trouton%27s_rule) to estimate the boiling point.

In [3]:
print(f"Estimated boiling point = {-avg_energy * 1e3 / 85:.1f} K")

Estimated boiling point = 72.3 K


## Osmotic second virial coefficient in the gas phase

We now use the same LJ parameters to estimate the osmotic second virial coefficient in the gas phase. Remember that $B_2$ describes exactly twobody interactions and can be used as a first order correction to the pressure of a non-ideal gas.
$$
B_2 = -2\pi \int_0^{\infty} \left ( e^{-u(r)/k_BT} - 1 \right ) r^2 dr
$$
The [experimental value](https://doi.org/10.1039/TF9676301320) is −298 cm$^3$/mol for $T=80.3$ K.

In [4]:
import numpy as np
from scipy.integrate import quad
from scipy.constants import gas_constant as R

T = 80.3  # in K

def integrand(r):
    u = 4 * epsilon * 1e3 * ((sigma / r)**12 - (sigma / r)**6)
    return np.expm1(-u / (R * T)) * r**2

integral, error = quad(integrand, 0, np.inf)
B2 = -2 * pi * integral * 1e-24 * N_A # in cm3 / mol

print(f"Estimated B2 at {T} K = {B2:.1f} cm³/mol")

Estimated B2 at 80.3 K = -363.9 cm³/mol


## Advanced tasks

1. Use experimental $B_2(T)$ from the article above to fit LJ $\sigma$ and $\epsilon$ over many temperatures.
2. Make global fit on both heat of vaporization and $B_2(T)$ to produce a model that captures both.
3. Estimate $\epsilon$ from QM calculation of the electronic polarization and ionization energy.
4. Use QM to calculate $u(r)$ exactly. How does this reproduce thermodynamics? Does it fit a LJ potential?